In [1]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=2, minor=7, micro=12, releaselevel='final', serial=0)
2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]


In [21]:
# Import dataset
# Python 2
# pip install jieba
# pip install pyldavis

# -*- coding:utf-8 -*-

import pandas as pd
df = pd.read_csv("AfterSegmentation_space0.1.txt", header=None)
df.head()

,0
0,老 用户 有 特权 ， 恭喜 您 获得 “ 光纤 宽带 不 加价 升 100M ” 资格 。...
1,近期 ， 有 市民 手机 接到 内容 涉及 含 同学聚会 、 揭发 隐私 等 内容 要求 点...
2,【 您 有 一份 60G 流量 大礼 未 领取 】 亲 ， 开通 4G 功能 31 天内 可...
3,尊敬 的 客户 ： 您 已 成功 开通 中国电信 “ 总机 服务 ” 业务 ， 你 的 短 ...
4,平安 豪礼 提前 送 ， 苹果 7 送到 手 。 欢 GO 大礼 场 ， iPhone ...


In [22]:
df.shape

(178427, 1)

In [23]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
df["content_cutted"] = df#.apply(chinese_word_cut)

df.content_cutted.head()

0    老 用户 有 特权 ， 恭喜 您 获得 “ 光纤 宽带 不 加价 升 100M ” 资格 。...
1    近期 ， 有 市民 手机 接到 内容 涉及 含 同学聚会 、 揭发 隐私 等 内容 要求 点...
2    【 您 有 一份 60G 流量 大礼 未 领取 】 亲 ， 开通 4G 功能 31 天内 可...
3    尊敬 的 客户 ： 您 已 成功 开通 中国电信 “ 总机 服务 ” 业务 ， 你 的 短 ...
4    平安 豪礼 提前 送 ， 苹果 7 送到 手 。 欢 GO 大礼 场 ， iPhone   ...
Name: content_cutted, dtype: object

In [24]:
df.content_cutted.shape

(178427,)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 5
# n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0
                               )
lda.fit(tf)

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)